# 학습 가이드 에이전트

사용자가 배우고 싶은 주제를 입력하면 단계별 학습 계획을 자동으로 생성하는 에이전트입니다.

In [1]:
# 필요한 패키지 설치
%pip install langchain langchain-openai langchain-community tavily-python python-dotenv python-docx -q

Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
from pathlib import Path
from dotenv import load_dotenv
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_openai import ChatOpenAI
from langchain.agents import AgentExecutor, create_openai_tools_agent
from langchain import hub

# .env 파일 경로 설정
env_path = Path.cwd() / ".env"
if not env_path.exists():
    env_path = Path("uuyeong/.env")
    if not env_path.exists():
        env_path = Path(__file__).parent / ".env" if "__file__" in globals() else Path.cwd() / "uuyeong/.env"

# .env 파일에서 환경변수 로드
load_dotenv(dotenv_path=env_path)
print(f".env 파일 경로: {env_path.absolute()}")

# API 키 확인
if "TAVILY_API_KEY" not in os.environ or os.environ["TAVILY_API_KEY"] == "YOUR_KEY":
    print("경고: TAVILY_API_KEY 환경변수가 설정되지 않았습니다.")
    print("uuyeong/.env 파일에서 TAVILY_API_KEY를 설정해주세요.")
else:
    print("Tavily API 키가 설정되었습니다.")
    print(f"API 키 (처음 10자): {os.environ['TAVILY_API_KEY'][:10]}...")

if "OPENAI_API_KEY" not in os.environ or os.environ["OPENAI_API_KEY"] == "YOUR_KEY":
    print("경고: OPENAI_API_KEY 환경변수가 설정되지 않았습니다.")
    print("uuyeong/.env 파일에서 OPENAI_API_KEY를 설정해주세요.")
else:
    print("OpenAI API 키가 설정되었습니다.")
    print(f"API 키 (처음 10자): {os.environ['OPENAI_API_KEY'][:10]}...")


/opt/miniconda3/envs/hateslop/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


.env 파일 경로: /Users/mac/Desktop/hateslop/hateslop_hackathon/.env
Tavily API 키가 설정되었습니다.
API 키 (처음 10자): tvly-dev-d...
OpenAI API 키가 설정되었습니다.
API 키 (처음 10자): sk-proj-x2...


## 학습 가이드 에이전트 설정


In [3]:
# Tavily 검색 Tool 생성 (학습 자료 검색용)
tavily_tool_guide = TavilySearchResults(
    api_key=os.environ.get("TAVILY_API_KEY", "YOUR_KEY"),
    max_results=10  # 학습 자료를 더 많이 수집
)

print("Tavily 검색 Tool이 생성되었습니다.")
print(f"최대 검색 결과 수: {tavily_tool_guide.max_results}")


Tavily 검색 Tool이 생성되었습니다.
최대 검색 결과 수: 10


/var/folders/xt/6hqj9_8d223c_wq515gdl8m80000gn/T/ipykernel_12767/701134673.py:2: LangChainDeprecationWarning: The class `TavilySearchResults` was deprecated in LangChain 0.3.25 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-tavily package and should be used instead. To use it run `pip install -U :class:`~langchain-tavily` and import as `from :class:`~langchain_tavily import TavilySearch``.
  tavily_tool_guide = TavilySearchResults(


## 학습 가이드 에이전트 생성

프롬프트 엔지니어링 기법을 활용하여 구조화된 학습 계획을 생성합니다.


In [4]:
from datetime import datetime
import json
import re

# LLM 초기화
llm_guide = ChatOpenAI(model="gpt-4-turbo", temperature=0)

# 기본 프롬프트 가져오기 (agent_scratchpad 포함)
base_prompt = hub.pull("hwchase17/openai-functions-agent")

# 시스템 메시지 수정 (Role Prompting + Few-shot + CoT)
# 주의: JSON 예시의 중괄호는 {{ }}로 이스케이프해야 함
custom_system_message = """너는 경험이 풍부한 교육 전문가이자 학습 설계자야.

사용자가 배우고 싶은 주제에 대해 다음 원칙을 따라 학습 가이드를 작성해:

1. **단계별 구성**: 기초 → 중급 → 고급 순서로 단계를 나눠
2. **실용적인 시간 배분**: 초보자 기준으로 각 단계에 적절한 시간 배분
3. **검증된 자료 추천**: 최신이면서도 검증된 교재와 사이트 추천
4. **구체적인 투두리스트**: 각 단계마다 체크 가능한 구체적인 할 일 목록

[출력 형식]
반드시 다음 JSON 형식으로 출력해:
{{
  "topic": "주제명",
  "total_duration_days": 총_학습_일수,
  "start_date": "시작일 (YYYY-MM-DD)",
  "end_date": "종료일 (YYYY-MM-DD)",
  "steps": [
    {{
      "step_number": 1,
      "title": "단계 제목",
      "duration_days": 단계별_일수,
      "start_date": "시작일 (YYYY-MM-DD)",
      "end_date": "종료일 (YYYY-MM-DD)",
      "learning_content": ["학습 내용 1", "학습 내용 2"],
      "recommended_books": ["교재 1", "교재 2"],
      "recommended_sites": [
        {{"name": "사이트명", "url": "URL"}}
      ],
      "todos": ["투두 1", "투두 2", "투두 3"]
    }}
  ]
}}

중요: JSON 형식만 출력하고, 추가 설명은 하지 마."""

# 프롬프트 메시지 수정 (시스템 메시지 교체)
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

# 기본 프롬프트의 구조를 유지하면서 시스템 메시지만 교체
learning_guide_prompt = ChatPromptTemplate.from_messages([
    ("system", custom_system_message),
    MessagesPlaceholder(variable_name="chat_history", optional=True),
    ("human", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad"),
])

# Agent 생성
tools_guide = [tavily_tool_guide]
agent_guide = create_openai_tools_agent(llm_guide, tools_guide, learning_guide_prompt)
agent_executor_guide = AgentExecutor(agent=agent_guide, tools=tools_guide, verbose=False)  # verbose=False로 설정하여 출력 간소화

print("학습 가이드 에이전트가 생성되었습니다.")
print(f"사용 가능한 Tool: {[tool.name for tool in tools_guide]}")


학습 가이드 에이전트가 생성되었습니다.
사용 가능한 Tool: ['tavily_search_results_json']


## 학습 가이드 생성 함수


### 학습 가이드 생성 및 파싱


In [5]:
def create_learning_guide(topic: str, start_date: str = None):
    """
    학습 가이드를 생성하는 함수
    
    Args:
        topic: 학습하고 싶은 주제
        start_date: 시작 날짜 (YYYY-MM-DD 형식, None이면 오늘)
    
    Returns:
        학습 가이드 딕셔너리
    """
    if start_date is None:
        start_date = datetime.now().strftime('%Y-%m-%d')
    
    # Agent 실행
    query = f"'{topic}'를 배우고 싶어. 단계별 학습 계획을 만들어줘. 오늘은 {start_date}야."
    
    print(f"📚 '{topic}' 학습 가이드 생성 중...")
    print("="*60)
    
    result = agent_executor_guide.invoke({"input": query})
    output = result["output"]
    
    print("\n✅ 검색 완료! 학습 가이드를 생성하는 중...\n")
    
    # JSON 추출 시도
    try:
        # 마크다운 코드 블록에서 JSON 추출
        if "```json" in output:
            json_str = output.split("```json")[1].split("```")[0].strip()
        elif "```" in output:
            json_str = output.split("```")[1].split("```")[0].strip()
        else:
            # JSON 부분만 찾기
            json_match = re.search(r'\{.*\}', output, re.DOTALL)
            if json_match:
                json_str = json_match.group(0)
            else:
                raise ValueError("JSON을 찾을 수 없습니다.")
        
        guide = json.loads(json_str)
        print("✅ 학습 가이드가 성공적으로 생성되었습니다!\n")
        return guide
    except Exception as e:
        print(f"❌ JSON 파싱 실패: {e}")
        print("\n원본 출력:")
        print(output)
        return {"raw_output": output, "error": str(e)}

print("학습 가이드 생성 함수가 준비되었습니다.")


학습 가이드 생성 함수가 준비되었습니다.


### 학습 가이드 출력 함수


In [6]:
def print_learning_guide(guide: dict):
    """학습 가이드를 보기 좋게 출력"""
    if "error" in guide:
        print(guide["raw_output"])
        return
    
    print("\n" + "="*60)
    print(f"📖 학습 주제: {guide.get('topic', 'N/A')}")
    print(f"📅 학습 기간: {guide.get('start_date', 'N/A')} ~ {guide.get('end_date', 'N/A')}")
    print(f"⏱️  총 학습 일수: {guide.get('total_duration_days', 'N/A')}일")
    print("="*60)
    
    for step in guide.get("steps", []):
        print(f"\n{'='*60}")
        print(f"📌 {step.get('step_number', 'N/A')}단계: {step.get('title', 'N/A')}")
        print(f"   기간: {step.get('start_date', 'N/A')} ~ {step.get('end_date', 'N/A')} ({step.get('duration_days', 'N/A')}일)")
        print(f"\n   📚 학습 내용:")
        for content in step.get("learning_content", []):
            print(f"      • {content}")
        
        if step.get("recommended_books"):
            print(f"\n   📖 추천 교재:")
            for book in step.get("recommended_books", []):
                print(f"      • {book}")
        
        if step.get("recommended_sites"):
            print(f"\n   🌐 참고 사이트:")
            for site in step.get("recommended_sites", []):
                if isinstance(site, dict):
                    print(f"      • {site.get('name', 'N/A')}: {site.get('url', 'N/A')}")
                else:
                    print(f"      • {site}")
        
        if step.get("todos"):
            print(f"\n   ✅ 투두리스트:")
            for todo in step.get("todos", []):
                print(f"      ☐ {todo}")

print("학습 가이드 출력 함수가 준비되었습니다.")


학습 가이드 출력 함수가 준비되었습니다.


### 투두리스트 생성 함수


In [7]:
def generate_todo_markdown(guide: dict) -> str:
    """학습 가이드에서 마크다운 형식의 투두리스트 생성"""
    if "error" in guide:
        return "가이드가 생성되지 않았습니다."
    
    markdown = f"# {guide.get('topic', '학습')} 투두리스트\n\n"
    markdown += f"**학습 기간**: {guide.get('start_date', 'N/A')} ~ {guide.get('end_date', 'N/A')}\n"
    markdown += f"**총 학습 일수**: {guide.get('total_duration_days', 'N/A')}일\n\n"
    
    for step in guide.get("steps", []):
        step_title = f"{step.get('step_number', 'N/A')}. {step.get('title', 'N/A')}"
        step_date = f"{step.get('start_date', 'N/A')} ~ {step.get('end_date', 'N/A')}"
        
        markdown += f"## {step_title}\n"
        markdown += f"**기간**: {step_date}\n\n"
        
        for todo in step.get("todos", []):
            markdown += f"- [ ] {todo}\n"
        
        markdown += "\n"
    
    return markdown

print("투두리스트 생성 함수가 준비되었습니다.")


투두리스트 생성 함수가 준비되었습니다.


### 워드 파일 생성 함수


In [8]:
from docx import Document
from docx.shared import Pt, RGBColor, Inches
from docx.enum.text import WD_ALIGN_PARAGRAPH
from datetime import datetime

def save_learning_guide_to_word(guide: dict, filename: str = None) -> str:
    """
    학습 가이드를 워드 파일로 저장하는 함수
    
    Args:
        guide: 학습 가이드 딕셔너리
        filename: 저장할 파일명 (None이면 자동 생성)
    
    Returns:
        저장된 파일 경로
    """
    if "error" in guide:
        print("❌ 가이드가 생성되지 않아 워드 파일을 만들 수 없습니다.")
        return None
    
    # 파일명 생성
    if filename is None:
        topic = guide.get('topic', '학습가이드').replace(' ', '_')
        timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
        filename = f"{topic}_학습가이드_{timestamp}.docx"
    
    # 워드 문서 생성
    doc = Document()
    
    # 제목 스타일 설정
    title = doc.add_heading(f"{guide.get('topic', '학습 가이드')} 학습 가이드", 0)
    title.alignment = WD_ALIGN_PARAGRAPH.CENTER
    
    # 기본 정보
    doc.add_paragraph(f"학습 기간: {guide.get('start_date', 'N/A')} ~ {guide.get('end_date', 'N/A')}")
    doc.add_paragraph(f"총 학습 일수: {guide.get('total_duration_days', 'N/A')}일")
    doc.add_paragraph("")  # 빈 줄
    
    # 각 단계별 내용
    for step in guide.get("steps", []):
        # 단계 제목
        step_title = doc.add_heading(
            f"{step.get('step_number', 'N/A')}단계: {step.get('title', 'N/A')}", 
            level=1
        )
        
        # 기간 정보
        period_para = doc.add_paragraph()
        period_para.add_run(f"기간: ").bold = True
        period_para.add_run(
            f"{step.get('start_date', 'N/A')} ~ {step.get('end_date', 'N/A')} "
            f"({step.get('duration_days', 'N/A')}일)"
        )
        doc.add_paragraph("")  # 빈 줄
        
        # 학습 내용
        if step.get("learning_content"):
            doc.add_paragraph("학습 내용:", style='Heading 2')
            for content in step.get("learning_content", []):
                para = doc.add_paragraph(content, style='List Bullet')
            doc.add_paragraph("")  # 빈 줄
        
        # 추천 교재
        if step.get("recommended_books"):
            doc.add_paragraph("추천 교재:", style='Heading 2')
            for book in step.get("recommended_books", []):
                para = doc.add_paragraph(book, style='List Bullet')
            doc.add_paragraph("")  # 빈 줄
        
        # 참고 사이트
        if step.get("recommended_sites"):
            doc.add_paragraph("참고 사이트:", style='Heading 2')
            for site in step.get("recommended_sites", []):
                if isinstance(site, dict):
                    para = doc.add_paragraph()
                    para.add_run(f"{site.get('name', 'N/A')}: ").bold = True
                    para.add_run(site.get('url', 'N/A'))
                else:
                    doc.add_paragraph(str(site), style='List Bullet')
            doc.add_paragraph("")  # 빈 줄
        
        # 투두리스트
        if step.get("todos"):
            doc.add_paragraph("투두리스트:", style='Heading 2')
            for todo in step.get("todos", []):
                para = doc.add_paragraph(todo, style='List Bullet')
            doc.add_paragraph("")  # 빈 줄
        
        # 단계 구분선
        doc.add_paragraph("─" * 50)
        doc.add_paragraph("")  # 빈 줄
    
    # 파일 저장
    doc.save(filename)
    print(f"✅ 워드 파일이 저장되었습니다: {filename}")
    return filename

print("워드 파일 생성 함수가 준비되었습니다.")


워드 파일 생성 함수가 준비되었습니다.


## 사용 예시

아래 셀에서 원하는 주제를 입력하여 학습 가이드를 생성할 수 있습니다.


In [9]:
# 여기에 원하는 학습 주제를 입력하세요
my_topic = "머신러닝"  # 예: "Python 프로그래밍", "웹 개발", "데이터 분석", "뜨개질", "C언어" 등

# 학습 가이드 생성
my_guide = create_learning_guide(my_topic)

# 결과 출력 (간단히)
if "error" not in my_guide:
    print(f"\n✅ '{my_guide.get('topic', 'N/A')}' 학습 가이드가 생성되었습니다!")
    print(f"📅 학습 기간: {my_guide.get('start_date', 'N/A')} ~ {my_guide.get('end_date', 'N/A')}")
    print(f"⏱️  총 학습 일수: {my_guide.get('total_duration_days', 'N/A')}일")
    print(f"📌 총 {len(my_guide.get('steps', []))}단계로 구성됩니다.\n")
    
    # 워드 파일로 저장
    word_file = save_learning_guide_to_word(my_guide)
    print(f"\n📄 전체 내용은 워드 파일에서 확인하세요: {word_file}")
else:
    print("❌ 학습 가이드 생성에 실패했습니다.")
    print(my_guide.get("raw_output", ""))


📚 '머신러닝' 학습 가이드 생성 중...

✅ 검색 완료! 학습 가이드를 생성하는 중...

✅ 학습 가이드가 성공적으로 생성되었습니다!


✅ '머신러닝' 학습 가이드가 생성되었습니다!
📅 학습 기간: 2025-12-04 ~ 2026-03-03
⏱️  총 학습 일수: 90일
📌 총 3단계로 구성됩니다.

✅ 워드 파일이 저장되었습니다: 머신러닝_학습가이드_20251204_155811.docx

📄 전체 내용은 워드 파일에서 확인하세요: 머신러닝_학습가이드_20251204_155811.docx
